In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import random
from matplotlib import animation
import matplotlib.animation as animation
%matplotlib notebook
np.set_printoptions(precision=3)
random.seed(1)

# GIBBS SAMPLING IN CONDITIONAL RANDOM FIELDS
### Loic Landrieu, 2019

We consider an image $I$ whose structure is given by the graph $(V,E)$ where $V$ is the set of pixels and $E \subset V \times V$ encodes their adjacency (4 pixel adjacency - up, down, left right).

We want to retrieve a binary latent variable $Z \rightarrow \{0,1\}^V$ for each pixel. We have an observation $x \in \mathbb{\Omega}^V$ with $\Omega \subset \mathbb{R}^d$, related to the unobserved latent variable. However, we also know that the latent variable is spatially regular over the image.

To implement this prior, we model the random variable $Z$ as a conditional random field (CRF), which is a discriminative undirected graphical model.

<img src="CRF.png" alt="CRF" style="width: 200px;"/>

This model can be described by an Ising model:
$$
p(z \mid x ; \eta) = \exp\left(\sum_{i \in V} \eta_{i}z_i + \sum_{(i,j) \in E} \eta_{i,j}z_i z_j\right)~,
$$
with $\eta$ defined as:

$$
\eta_{i,j} = 4 \lambda,
$$
and
$$
\eta_i = \log\left( \frac{{q}_i}{1-{q}_i} \right) - 2 \lambda \mid N_i \mid,
$$
where $N_i$ denotes the set of pixel neighboring $i$, $\lambda\in\mathbb{R}$ is the regularization strength, and $q_i = p(z_i=1 \mid x_i)$.

In practice , we regularize $q$ to prevent numerical issues
$$
\hat{q_i} = (1-\alpha) q_i + \frac\alpha2
$$



Q1: Complete the following function which takes as input an array $\left( \pi_i \right)_{i \in V}$  of probabilities : $\pi_i = p(Y_i=1)$ for an unspecified random variable $Y$, and a regularizing parameter $\alpha$ and returns the array $\left(\hat{\pi}_i \right)_{i \in V}$
with $\hat{\pi}_i = (1-\alpha) \pi_i + \frac\alpha2$.

In [ ]:
def regularize(pis, alpha):
    return #TODO

Q2 In pratical applications probabilities $q_i$ are computed from the observations by another model. Here set them by hand in the cell below.


What can we expect the marginals $p(z_i = 1\mid x)$ to be like if $\lambda$ is high? Low? Extremely high?

In [ ]:
proba = 0.5*np.random.rand(8,8)
proba[0:2,:] = 0
proba[-2:,:] = 1
plt.figure()
plt.imshow(proba)
plt.set_cmap('binary')
plt.title('Observation probability')
plt.show()

Q3: complete the following function to compute the unary potentials $\eta_i$. Use a regularization term $\alpha=0.05$.

Visualize the image of unaries and make sure it makes sense. How can the first and last two rows have identical conditonals but not uniform unaries?

In [ ]:
def compute_unary(proba, reg_strength):
    n_lin = proba.shape[0]
    n_col = proba.shape[1]
    n_nei = np.zeros((n_lin,n_col))
    unary = np.zeros((n_lin,n_col))
    
    #count the neighbours
    n_nei[1:,:] += 1
    n_nei[:-1,:] += 1
    n_nei[:,1:] += 1
    n_nei[:,:-1] += 1

    smoothed_proba = #TODO: regularized version of proba

    return #TODO eta

In [ ]:
proba = 0.5*np.random.rand((8,8))
proba[0:2,:] = 0
proba[-2:,:] = 1
plt.figure()
plt.imshow(proba)
plt.set_cmap('binary')
plt.title('Observation probability')
plt.show()

We use the Gibbs sampling algorithm to compute the pixel marginals of $X$ : $p(z_{i}=1\mid x;\eta)$.

$z^{(0)} \leftarrow \;\texttt{random initialization}$

$\texttt{for t = 1 to max_ite_gibbs:}$

$\quad i\leftarrow \texttt{random pixel}$
    
$\quad \texttt{sample}\; z^{(t)}_i\; \texttt{from}\; p(z_{i}\mid z^{(t-1)},x;\eta)$

$\quad z_{i}^{(t)} \leftarrow  z_{i}^{(t-1)}$

After enough iteration, we have:

$$p(z_{i}=1\mid x;\eta) \approx \frac1{T-T_0}\sum_{t = T_0+1}^T z_{i}^{(t)}.$$

Note: in the Ising model, we have:

$$
p(z_{i}\mid z^{(t-1)},x;\eta) = \frac1{\exp\left(-\eta_i - \sum_{j \sim i} \eta_{i,j} x_j\right)}
$$

Q4: Complete the following function to compute the interaction term $ \sum_{j \sim i} \eta_{i,j} x_j$ for all pixels. Visualize the results for a current sample $z^{(t)}$ taken at $1$ where $p(z_i^{(t)}\mid x_i)\geq0.5$. Make sure you understand the results.

In [ ]:
def compute_interaction(current, reg_strength):
    n_lin = current.shape[0]
    n_col = current.shape[1]
    #compute a padded version of current with extra zero values on the borders for efficient computation of the sum of x_j
    padded_current = np.zeros((n_lin+2,n_col+2))
    padded_current[1:-1,1:-1]= current
    sum_of_neighbors_x = (padded_current[0:-2,1:-1] + padded_current[2:,1:-1] + padded_current[1:-1,0:-2] + padded_current[1:-1,2:])
    return #TODO

In [ ]:
plt.figure()
plt.subplot(121)
plt.title('current')
plt.imshow(proba>=0.5)
plt.subplot(122)
plt.title('interaction')
plt.imshow(compute_interaction(proba>=0.5, 1))
plt.show()

Q5: complete the following function to compute one update step of Gibbs sampling.

In [ ]:
def one_gibbs_update(unary, current, interaction, reg_strength):
    n_lin = unary.shape[0]
    n_col = unary.shape[1]
    
    i_lin = np.random.randint(n_lin)
    i_col = np.random.randint(n_col)

    proba_conditional = #TODO

    new_value = np.random.rand()< proba_conditional
    
    #since we only changed the value of one pixel, we can only update the interatcion teerm of its neighbors 
    diff = new_value - current[i_lin, i_col]
    current[i_lin, i_col] = new_value

    if i_lin > 0 : interaction[i_lin-1, i_col] += #TODO
    if i_lin < n_lin - 1 : interaction[i_lin+1, i_col] += #TODO
    if i_col > 0 : interaction[i_lin, i_col-1] += #TODO
    if i_col < n_col - 1 : interaction[i_lin, i_col+1] += #TODO

In [ ]:
#test cell, run it a few times to make sure it doesn't crash

unary = compute_unary(proba, 0.5)
current = (np.random.randint(2, size = proba.shape)).astype('f4')
interaction = compute_interaction(current, 0.5)
for i in range(100):
    one_gibbs_update(unary, current, interaction, 0.5)

Q6: Complete the following function, which performs Gibbs sampling, keep track of the samples $\sum_{t = T_0+1}^T z_{i}^{(t)}$ and represent the marginals every $100$ iterations. Chose a burn in period of $10\%$ of the iterations.

In [ ]:
def animate_gibbs_simple(proba, n_ite_gibbs = 1000, reg_strength = 0.5):
    
    n_lin = proba.shape[0]
    n_col = proba.shape[1]
    burn_in = #TODO

    global sample_history, current, interaction
    current = (np.random.randint(2, size = proba.shape)).astype('f4')
    unary = #TODO
    interaction = #TODO
    sample_history = np.zeros_like(proba)
    
    fig = plt.figure()
    plt.title("Burn in") 
    plt.axis('off')
    plt.set_cmap('binary')
    im = plt.imshow(sample_history, animated=True, vmin = 0, vmax = 1)

    def updatefig(i):
        #i is the iteration
        global sample_history, current, interaction
        one_gibbs_update(unary, current, interaction, reg_strength)  
        
        if i > burn_in :
            sample_history = #TODO

        if i > burn_in and i % 100 == 0 :
            plt.title("Iteration = %d" % (i))
            im.set_array(sample_history/(i-burn_in+1))
        
        return im,
    
    anim = animation.FuncAnimation(fig, updatefig, interval=1, blit=True, frames = n_ite_gibbs, repeat = False)
    plt.show()
    return anim

Q7: run the following cell a few times. What do you notice from one run to another? Was this expected?

Comment on the convergence speed.

In [ ]:
%matplotlib notebook
p = animate_gibbs_simple(proba, reg_strength = 1, n_ite_gibbs = 2000)

Q8: We consider a binary checker-like partition of the grid into black $B$ and whites $W$ tiles of a chess board, as represented below. Show that 
$$
p(z_B \mid z^{(t-1)}, x; \eta) =  p(z_B \mid z_W^{(t-1)}, x; \eta),
$$

and symetrically

$$
p(z_W \mid z^{(t-1)}, x; \eta) =  p(z_W \mid z_B^{(t-1)}, x; \eta).
$$

In [ ]:
plt.figure()
plt.set_cmap('binary')
coords=np.ogrid[0:8,0:8]
selected_pixel=(coords[0]+coords[1])%2 == 0
plt.imshow(selected_pixel)
plt.show()

Q9 : As a consequence we can modify the Gibbs algorithm to alternate between sampling all white and black tiles. Complete the follwoing function to implement this. Justify why this should be faster.

In [ ]:
def block_gibbs_update(even_ite, unary, current, interaction, reg_strength):
    n_lin = unary.shape[0]
    n_col = unary.shape[1]
    
    coords=np.ogrid[0:n_lin,0:n_col]
    selected_pixel=(coords[0]+coords[1])%2 == even_ite

    proba_conditional = #TODO - proba_conditional for all selected pixels
    new_value = np.random.rand(proba_emission.size)< proba_conditional
    current[selected_pixel] = new_value
    interaction[:,:] = compute_interaction(current, reg_strength)

In [ ]:
#test cell, run it a few times to make sure it doesn't crash

unary = compute_unary(proba, 0.5)
current = (np.random.randint(2, size = proba.shape)).astype('f4')
interaction = compute_interaction(current, 0.5)
for i in range(100):
    block_gibbs_update(i%2, unary, current, interaction, 0.5)

In [ ]:
def animate_gibbs_block(proba, n_ite_gibbs = 1000, reg_strength = 0.1, temperature = 1):  
   #TODO. Same as animate_gibbs_simple but with block sampling
    return anim

Q9 : Run Block Gibbs sampling on this larger exemple. It should be much faster.

Try the following value for reg strength and comment:
$$0,0.2,0.5,1,10,-1$$

In [ ]:
proba = 0.5*np.random.rand((100,100))
proba[0:2,:] = 0
proba[-2:,:] = 1
animate_gibbs_block(proba, n_ite_gibbs = 2000, reg_strength = 1)

Q10: We now consider an application to object/background image segmentation. This time the probability comes from a discriminative learning algorithm (random forrest) trained on the RGB values of an image coupled with its ground truth segmentation. The trained algorithm gives for each pixel of a test image a probability score of being in state object/background. Run the following cells to represent this probability, and comment on the resulting probability.

In [ ]:
from PIL import Image

In [ ]:
fish_train_rgb = np.asarray(Image.open("fish1.jpg").convert('RGB'))
fish_train_tag_rgb = np.asarray(Image.open("fish1Tag.jpg").convert('RGB'))
fish_train_gt = ((fish_1_tag_rgb).sum(-1) > 200) #white enough
fish_test_rgb = np.asarray(Image.open("fish3.jpeg").convert('RGB'))

fish_train_vec = fish_train_rgb.reshape(-1, fish1_rgb.shape[-1])
fish_test_vec = fish_test_rgb.reshape(-1, fish2_rgb.shape[-1])
fish_train_vec_gt = fish1_gt.flatten()
fish_train_dim = fish_train_rgb.shape[0:2]
fish_test_dim = fish_test_rgb.shape[0:2]

figure(num=None, figsize=(12, 5), dpi=80, facecolor='w', edgecolor='k')
plt.subplot(131)
plt.axis('off')
imshow(fish_train_rgb)
plt.title('train image')
plt.subplot(132)
plt.axis('off')
plt.title('ground truth')
imshow(fish_train_gt)
plt.subplot(133)
plt.axis('off')
plt.title('test image')
imshow(fish_test_rgb)

rfe = RandomForestClassifier(max_depth=None, min_samples_split=2, random_state=0, n_estimators = 10)
rfe.fit(fish_train_vec, fish_train_vec_gt)
fish_proba = rfe.predict_proba(fish_test_vec)[:,1]
fish_proba = fish_proba.reshape((fish_test_dim))
figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
plt.subplot(121)
plt.axis('off')
plt.title('probability')
imshow(fish_proba)
plt.subplot(122)
plt.axis('off')
plt.title('thresholded at 0.5')
imshow(fish_proba>0.5)

Q11: Run Block gibbs sampling on the object probability given by the random forrest. Find an appropriate regularization strength.

In [ ]:
#TODO

Note that Gibbs sampling is NOT an efficient method of object segmentation. This exercice is an illustration of the method.
Object segmentation amounts to compute the MAP of this problem, which can be efficienctly computed using max-flow formulations as long as the interactions are attractives.